In [ ]:
#default_exp xtransformer

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from fastai.basics import *
from transformers_sandbox.core import *
from transformers_sandbox.layers import *
from transformers_sandbox.attention import *
from transformers_sandbox.transformer import LMMixin, EncDecMixin
from transformers_sandbox.config import ConfigBase, update_sig
from transformers_sandbox.experimental import *

# XTransformer

> Generic model for testing new building blocks. WIP...

## Helpers

In [ ]:
#export
def wrap_sublayer(sublayer:Module, method:str, d_model):
    """
    Wraps a sublayer with skip connection defined by method
    currently supported:
    * postnorm
    * prenorm
    * admin
    * rezero
    * ...
    """
    if method == 'postnorm':
        return PostNorm(d_model, Residual(sublayer))
    elif method == 'prenorm':
        return Residual(PreNorm(d_model, sublayer))
    elif method == 'admin':
        raise NotImplementedError
    elif method == 'rezero':
        return ReZero(sublayer)
    else:
        raise NotImplementedError(f'{method} is not valid wrapping method.')

## Bricks

> Architecture specific layers, blocks and containers.

### Encoder

In [ ]:
#export
class XEncoderBlock(Module):
    """
    Experimental encoder block
    """
    def __init__(self,
                 d_model:int, 
                 n_heads:int = 8,
                 attn_module:Module = Attention,
                 ff_module:Module = FeedForward,
                 d_ff:int = None, 
                 attn_dropout:float = 0.1,
                 ff_dropout:float = 0.1,
                 causal:bool = False, 
                 attn_bias:bool = False, 
                 residual_type:str = 'postnorm',
                 shared_qk:bool=False,
                 **kwargs):
        store_attr('attn_dropout') # mb separate argument attn_post_dropout
        attn = attn_module(d_model, n_heads=n_heads, causal=causal, dropout=attn_dropout, bias=attn_bias, shared_qk=shared_qk)
        ff = ff_module(d_model, d_ff=d_ff, dropout=ff_dropout)
        self.attn = wrap_sublayer(attn, method=residual_type, d_model=d_model)
        self.ff = wrap_sublayer(ff, method=residual_type, d_model=d_model)
        
    def forward(self, x, mask=None): #? more args
        out = self.attn(x, mask=mask)
        return self.ff(out)

In [ ]:
bs = 4
sl = 128
d = 64
x = torch.randn(bs, sl, d)
m = XEncoderBlock(d)
out = m(x)
assert (out.size() == (bs, sl, d))
out.shape

torch.Size([4, 128, 64])

In [ ]:
#hide
m = XEncoderBlock(d, shared_qk=True)
out = m(x)
assert (out.size() == (bs, sl, d))

In [ ]:
#export
class XEncoder(Module):
    """Stack of XEncoderBlocks"""
    def __init__(self, 
                 d_model, 
                 n_layers=6, 
                 n_heads=8, 
                 d_ff=None,
                 attn_module:Module=Attention,
                 ff_module:Module=FeedForward,
                 ff_dropout=0.1, 
                 attn_dropout=0.1,
                 attn_bias=False,
                 causal=False, 
                 residual_type:str='postnorm',
                 shared_qk:bool=False,
                 final_norm=None,
                 **kwargs):
        store_attr('d_model')
        self.layers = nn.ModuleList([])    
        for _ in range(n_layers):
            self.layers.append(XEncoderBlock(d_model, n_heads, causal=causal, attn_module=attn_module,
                                    d_ff=d_ff, attn_dropout=attn_dropout, ff_dropout=ff_dropout, ff_module=ff_module,
                                    residual_type=residual_type, attn_bias=attn_bias, shared_qk=shared_qk,
                                            **kwargs))
        self.norm = None if final_norm is None else final_norm(d_model)
        
    def forward(self, x, mask=None):
        for layer in self.layers: x = layer(x, mask=mask)
        if self.norm is not None: x = self.norm(x)
        return x

In [ ]:
# postnorm
x = torch.randn(bs, sl, d)
m = XEncoder(d)
out = m(x)
assert (out.size() == (bs, sl, d))
out.shape

torch.Size([4, 128, 64])

In [ ]:
#rezero: all residual branches are zeroed out at initialization
m = XEncoder(d, residual_type='rezero')
out  = m(x)
assert (out.size() == (bs, sl, d))
assert (out == x).all()

### Decoder

In [ ]:
#export
class XDecoderBlock(Module):
    """
    Standart transformer decoder block. Consist of self-attention, encoder-decoder attention 
    and positiona feed-forward alyers
    """
    def __init__(self,
                 d_model, 
                 n_heads = 8, 
                 d_ff = None,
                 attn_dropout = 0.1, 
                 ff_dropout=0.1,
                 mask = None ,
                 attn_bias=False,
                 residual_type='postnorm'):
        # mb separate argument attn_post_dropout
        attn = Attention(d_model, n_heads=n_heads, causal=True, dropout=attn_dropout, bias=attn_bias)
        cross = Attention(d_model, n_heads=n_heads, causal=False, dropout=attn_dropout, bias=attn_bias)
        ff = FeedForward(d_model, d_ff=d_ff, dropout=ff_dropout)
        self.attn = wrap_sublayer(attn, method=residual_type, d_model=d_model)
        self.cross = wrap_sublayer(cross, method=residual_type, d_model=d_model)
        self.ff = wrap_sublayer(ff, method=residual_type, d_model=d_model)
        
    def forward(self, x, context, mask=None, context_mask=None):
        out = self.attn(x, mask=mask)
        out = self.cross(out, context, mask=mask, context_mask=context_mask)
        return self.ff(out)

In [ ]:
#export
class XDecoderBlockV2(Module):
    """Transformer decoder block using additive attention layer instead of self-attention 
    followed by cross-attention"""
    def __init__(self,
                 d_model, 
                 n_heads = 8, 
                 mask = None, 
                 d_ff=None,
                 attn_dropout=0.1, 
                 ff_dropout=0.1, 
                 attn_bias=False,
                 residual_type='postnorm'):
        attn = AdditiveAttention(d_model, n_heads=n_heads, causal=True, dropout=attn_dropout, bias=attn_bias)
        ff = FeedForward(d_model, d_ff=d_ff, dropout=ff_dropout)
        self.attn = wrap_sublayer(attn, method=residual_type, d_model=d_model)
        self.ff = wrap_sublayer(ff, method=residual_type, d_model=d_model)
        
    def forward(self, x, context, mask=None, context_mask=None):
        out = self.attn(x, context, mask=mask, context_mask=context_mask)
        out = self.ff(out)
        return out

In [ ]:
#export
class XDecoder(Module):
    """Stack of TransformerDecoder layers"""
    def __init__(self, 
                 d_model, 
                 n_layers=6, 
                 n_heads=8, 
                 d_ff=None, 
                 attn_dropout=0.1, 
                 ff_dropout=0.1, 
                 residual_type='postnorm', 
                 comb_attn=False, 
                 attn_bias=False, 
                 final_norm=None):
        store_attr('d_model')
        #TODO(Arto) refactor
        block = XDecoderBlockV2 if comb_attn else XDecoderBlock
        self.layers = nn.ModuleList([])
        for _ in range(n_layers):
            self.layers.append(block(d_model, n_heads, d_ff=d_ff, attn_dropout=attn_dropout, 
                                     ff_dropout=ff_dropout, residual_type=residual_type, attn_bias=attn_bias))
        self.norm = None if final_norm is None else final_norm(d_model)
        
    def forward(self, x, context, mask=None, context_mask=None):
        for layer in self.layers: x = layer(x, context, mask, context_mask)
        if self.norm is not None: x = self.norm(x)
        return x

In [ ]:
x = torch.randn(bs, sl, d)
context = torch.randn(bs, sl, d)
m = XDecoder(d)
out = m(x, context)
assert (out.size() == (bs, sl, d))
out.shape

torch.Size([4, 128, 64])

## Models

### Language model

In [ ]:
#export
class XTransformerLM(Module, LMMixin):
    """
    Basic Transformer for language modelling
    
    Parameters:
        * vocab_sz: int
        * d_model: int - inner dimension of the model
        * n_layers: int (default: 6) 
        * n_heads: int (default: 8)
        * d_ff: int - inner dimension of the pointwise FeedForward net, if None defaults to 4*d_model
        * attn_dropout: float - attention dropout
        * ff_dropout: float - feed-forward dropout
        * emb_dropout: float - embedding dropout
        * causal: bool (default: True) - if True does causal masking automatically
        * max_seq_len: int (default: 512)
        * tie_weights: bool - if True target embedding weights are used for computation output projection
        * residual_type: str - one of {'postnorm', 'prenorm', 'admin', 'rezero'}
        * attn_bias: bool - wether to allow biases in attention projection layers
        * pad_idx: int - padding token id, required for autogeneration of padding mask
        * pos_enc: str from {'absolute', 'fixed', 'axial'} - type of positional encoding to use
        * axial_shape: tuple - [optional] should be factors of max_seq_len
        * axial_emb_dims: tuple - [optional] axial embedding components, should sum to d_model
    Inputs:
        * x - input ids, shape [bs, sl]
        * mask - optional boolean mask, shape [bs, sl]
    Returns:
        * logits - target token logits, shape [bs, sl, vocab_sz]
    """
    def __init__(self, 
                 vocab_sz:int, 
                 d_model:int, 
                 n_layers:int=6,
                 n_heads:int=8,
                 d_ff:int=None,
                 attn_module:Module=Attention,
                 ff_module:Module=FeedForward,
                 attn_dropout:float=0.1,
                 ff_dropout:float=0.1,
                 emb_dropout:float=0.1,
                 tie_weights:bool=True,
                 causal:bool=True,
                 pos_enc:str='absolute',
                 max_seq_len:int=512,
                 axial_shape:tuple=None,
                 axial_emb_dims:tuple=None,
                 pad_idx:int=None,
                 residual_type:str='postnorm',
                 attn_bias:bool=False,
                 shared_qk:bool=False):
        store_attr()
        self.emb = TransformerEmbedding(vocab_sz, d_model, max_seq_len, dropout=emb_dropout, 
                                        pos_enc=pos_enc, axial_shape=axial_shape, 
                                        axial_emb_dims=axial_emb_dims)
        final_norm = nn.LayerNorm if (residual_type in {'prenorm'}) else None
        self.encoder = XEncoder(d_model, n_layers, n_heads, causal=causal, d_ff=d_ff,
                                attn_module=attn_module, ff_module=ff_module,
                                attn_dropout=attn_dropout, ff_dropout=ff_dropout,
                                residual_type=residual_type, attn_bias=attn_bias,
                                shared_qk=shared_qk, final_norm=final_norm)
        self.proj = nn.Linear(d_model, vocab_sz)
        if tie_weights: self.proj.weight = self.emb.emb.weight
        
    def forward(self, x, mask=None):
        x = self.emb(x)
        x = self.encoder(x, mask=mask)
        return self.proj(x)


In [ ]:
bs = 4
sl = 128
d = 64
vocab_sz = 256
x = torch.randint(vocab_sz, (bs, sl))
model = XTransformerLM(vocab_sz, d, n_layers=2, causal=False)
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))
out.shape

torch.Size([4, 128, 256])

In [ ]:
#hide
model = XTransformerLM(vocab_sz, d, n_layers=2, causal=True, residual_type='prenorm')
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))

In [ ]:
#TODO
#add tests for various configs here

In [ ]:
#hide
# model = XTransformerLM(vocab_sz, d, n_layers=2, causal=True, shared_qk=True)
# out = model(x)
# assert (out.size() == (bs, sl, vocab_sz))
# assert isinstance(model.encoder.layers[0].attn.sublayer.sublayer.in_proj, SharedQKAttnInProj)

In [ ]:
#hide
# def xtransformer_lm_splits(model):
#     "Splits TransformerLM `model` into groups for differential learning rates."
#     groups = L([model.emb] + [l for l in model.encoder.layers] + [model.proj])
#     return groups.map(params)

In [ ]:
#hide
# assert len(transformer_lm_splits(model)) == 2+2

### Encoder-Decoder model

> Warning: Not implemented yet

In [ ]:
#export
class XTransformer(Module, EncDecMixin):
    """
    Basic Transformer Encoder-Decoder model
    Parameters:
        * enc_vocab_sz: int - source vocab size 
        * dec_vocab_sz: int - target vocab size
        * d_model: int - inner dimension of the model
        * n_enc_layers: int (default: 6) 
        * n_dec_layers: int (default: 6) 
        * heads: int (default: 8)
        * d_ff: int - inner dimension of the pointwise FeedForward net, if None defaults to 4*d_model
        * attn_dropout: float - attention dropout
        * ff_dropout: float - feed-forward dropout
        * emb_dropout: float - embedding dropout
        * max_seq_len: int (default: 512)
        * prenorm: bool - whether to use PreNorm or PostNorm
        * attn_bias: bool - whether to allow biases in attention projection layers
        * pad_idx: int - padding token id, if pad_idx is provided, and no mask/context_mask are 
                passed to forward method will be used to generate padding masks
        * tie_weights: bool - if True target embedding weights are used for computation output projection
        * shared_emb: bool - if True encoder and decoder will use shared embedding layer
        * pos_enc: str from {'absolute', 'fixed', 'axial'} - type of positional encoding to use
        * axial_shape: tuple - [optional] should be factors of max_seq_len
        * axial_emb_dims: tuple - [optional] axial embedding components, should sum to d_model
    Inputs:
        * src - source input ids, shape [bs, src_sl]
        * tgt - target input ids, shape [bs, tgt_sl]
        * src_mask - optional boolean source mask, shape [bs, src_sl]
        * tgt_mask - optional boolean target mask, shape [bs, tgt_sl]
    Returns:
        * logits - target token logits, shape [bs, tgt_sl, tgt_vocab_sz]
    """
    def __init__(self, 
                 enc_vocab_sz, 
                 dec_vocab_sz, 
                 d_model, 
                 n_enc_layers=6, 
                 n_dec_layers=6, 
                 n_heads=8, 
                 d_ff=None,
                 pad_idx=None, 
                 tie_weights=True,
                 shared_emb = False,
                 attn_dropout=0.1, 
                 ff_dropout=0.1, 
                 emb_dropout=0.1,
                 prenorm=False, 
                 attn_bias=False,
                 comb_attn=False, 
                 pos_enc='absolute', 
                 max_seq_len=512, 
                 axial_shape=None, 
                 axial_emb_dims=None):
        store_attr()
        self.enc_emb = TransformerEmbedding(enc_vocab_sz, d_model, max_seq_len, dropout=emb_dropout, pos_enc=pos_enc,
                                            axial_shape=axial_shape, axial_emb_dims=axial_emb_dims)
        if shared_emb:
            assert (enc_vocab_sz == dec_vocab_sz), "Encoder and decoder vocab size doesn't match"
            self.dec_emb = self.enc_emb
        else:
            self.dec_emb = TransformerEmbedding(dec_vocab_sz, d_model, max_seq_len, dropout=emb_dropout, pos_enc=pos_enc,
                                                axial_shape=axial_shape, axial_emb_dims=axial_emb_dims)
        final_norm = nn.LayerNorm if prenorm else None
        self.encoder = TransformerEncoder(d_model, n_enc_layers, n_heads, d_ff=d_ff, attn_dropout=attn_dropout, ff_dropout=ff_dropout, 
                                          prenorm=prenorm, attn_bias=attn_bias, final_norm=final_norm, causal=False)
        self.decoder = TransformerDecoder(d_model, n_dec_layers, n_heads, d_ff=d_ff, attn_dropout=attn_dropout, ff_dropout=ff_dropout, 
                                          prenorm=prenorm, comb_attn=comb_attn, attn_bias=attn_bias, final_norm=final_norm)
        self.proj = nn.Linear(d_model, dec_vocab_sz)
        if tie_weights: self.proj.weight = self.dec_emb.emb.weight

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        src_mask = default(src_mask, self.get_padding_mask(src))
        tgt_mask = default(tgt_mask, self.get_padding_mask(tgt))
        enc = self.encoder(self.enc_emb(src), mask=src_mask)
        out = self.decoder(self.dec_emb(tgt), context=enc, mask=tgt_mask, context_mask=src_mask)
        return self.proj(out)
    
    def get_padding_mask(self, x):
        if self.pad_idx is None: return None
        return (x != self.pad_idx)


In [ ]:
#hide
# bs = 4
# src_sl = 70
# tgt_sl = 80
# d = 64
# src_vocab_sz = 256
# tgt_vocab_sz = 256
# src = torch.randint(src_vocab_sz, (bs, src_sl))
# tgt = torch.randint(tgt_vocab_sz, (bs, tgt_sl))
# model = Transformer(src_vocab_sz, tgt_vocab_sz, d, n_enc_layers=2, n_dec_layers=2)
# out = model(src, tgt)
# assert (out.size() == (bs, tgt_sl, tgt_vocab_sz))
# out.shape

In [ ]:
#export
#TODO find out what is the best way to split encoder-decoder architecture
# def transformer_splits(model):
#     "[v0] Splits Transformer `model` into groups for differential learning rates."
#     groups = L([nn.ModuleList([model.enc_emb, model.dec_emb])] + [l for l in model.encoder.layers] + [l for l in model.decoder.layers] + [model.proj])
#     return groups.map(params)

In [ ]:
#hide
# assert len(transformer_splits(model)) == 2+4

## Config for experiments

In [ ]:
#export
class XConfig(ConfigBase):
    """
    Config for enwik8 Experiment.
    See https://arampacha.github.io/reformer_fastai/experiment.enwik8-baseline.html for details
    """
    _model = XTransformerLM
    _d = {
        'vocab_sz':256,
        'd_model':512,
        'n_layers':12,
        'n_heads':8,
        'attn_module':Attention,
        'ff_module':FeedForward,
        'd_ff':None,
        'attn_dropout':0.1,
        'ff_dropout':0.1,
        'emb_dropout':0.1,
        'tie_weights':True,
        'causal':True,
        'pos_enc':'absolute',
        'max_seq_len':512,
        'axial_shape':None,
        'axial_emb_dims':None,
        'pad_idx':None,
        'attn_bias':False,
        'shared_qk':False,
        'residual_type':'postnorm',
    }
    @update_sig(_d)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [ ]:
model = XTransformerLM.from_config(XConfig(n_layers=2, residual_type='rezero'))
model

XTransformerLM(
  (emb): TransformerEmbedding(
    (emb): Embedding(256, 512)
    (dropout): Dropout(p=0.1, inplace=False)
    (pos_enc): AbsolutePositionalEmbedding(
      (emb): Embedding(512, 512)
    )
  )
  (encoder): XEncoder(
    (layers): ModuleList(
      (0): XEncoderBlock(
        (attn): ReZero(
          (sublayer): Attention(
            (in_proj): AttnInProjV2(
              (to_q): Linear(in_features=512, out_features=512, bias=False)
              (to_kv): Linear(in_features=512, out_features=1024, bias=False)
            )
            (attn): ScaledDotProdAttention(
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (out_proj): Linear(in_features=512, out_features=512, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (scale): Scale()
        )
        (ff): ReZero(
          (sublayer): FeedForward(
            (net): Sequential(
              (fc1): Linear(in_features=512, out_features=2048, bias=Tru

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_attention.ipynb.
Converted 03_models.transformer.ipynb.
Converted 04a_models.reformer.ipynb.
Converted 04x_models.xtransformer.ipynb.
Converted 05_tokenizers.ipynb.
Converted 06_data.ipynb.
Converted 07_metrics.ipynb.
Converted 08_optimizers.ipynb.
Converted 09_tracking.ipynb.
Converted 10_config.ipynb.
Converted 40_experimental.ipynb.
Converted index.ipynb.
